In [ ]:
import pandas as pd 
import numpy as np
from sklearn import model_selection
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

For this report, I'll be working with Tidy Tuesday [data](https://github.com/rfordatascience/tidytuesday/tree/master/data/2022/2022-09-13) about the important topic of reported Bigfoot sightings. 

Now, I don't think the evidence supports Bigfoot's existence any more strongly than the Easter Bunny's, but the data should be interesting regardless.

In [ ]:
raw_data = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2022/2022-09-13/bigfoot.csv")
raw_data.head() 
raw_data.shape

The Bigfoot Field Researchers Organization did their work well: each sighting comes with an impressive amount of metadata. Unfortunately, there are 
lots of NaN.

In [ ]:
raw_data.columns
raw_data.isnull().sum()

We may as well convert the date column to the proper data type.

In [ ]:
raw_data["date"] = pd.to_datetime(raw_data["date"])

We will predict the region of reported sightings from other variables. 

# Available Features

The "classification" column refers to the 
plausibility of the sighting, with "C" denoting 
second- or third-hand reports. (See [https://www.bfro.net/GDB/classify.asp](https://www.bfro.net/GDB/classify.asp))

In [ ]:
y = raw_data["state"]
X = raw_data
del raw_data["state"]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = .25, random_state = 12345)

# EDA 

Some states have only a few observations (what is Bigfoot doing in Delaware?), which will make prediction more difficult. As we'd expect, most come from the Pacific Northwest.

In [ ]:
y_train.value_counts()

Sightings span a long time range.

In [ ]:
gaussian_kde(X_train.dropna()["date"].dt.year)

Does the distribution of classification types 
vary by state? Overall, the split is about even between A and B.
In some states, class A reports (the most credible) predominate.

In [ ]:
X_train["classification"].value_counts()
X_train["classification"].groupby([y_train]).value_counts()

A simple longitude-latitude plot shows a surprising number of sightings on the East Coast. Bigfoot is usually thought of escaping detection in the sparsely settled West.
But if we assume all Bigfoot sightings are false or mistaken, we would expect sighting density to correlate with population. That would explain the eastern sightings.

In [ ]:
# see 
X_train.plot(x = "longitude", y = "latitude", kind = "scatter")
plt.show()

Several of the variables have normal-ish distributions.

In [ ]:
def plot_density(var): 
    X_train[var].plot.kde()
    plt.title(var)
    plt.show()

for var in ["moon_phase", "temperature_mid", "dew_point", "humidity"]: 
    plot_density(var)

I recode states into regions, rather arbitrarily.  

In [ ]:
regions = {"mid_atlantic" : ["Delaware", "Maryland", "Virginia", "West Virginia", "New Jersey" , "New York", "Pennsylvania"],
"southeast" : ["South Carolina", "North Carolina", "Georgia", "Tennessee", "Kentucky", "Arkansas", "Louisiana", "Alabama" , "Mississippi", "Florida"],
"new_england" : ["Rhode Island", "Vermont", "New Hampshire", "Maine", "Connecticut", "Massachusetts"], 
"midwest" : ["Ohio", "Illinois", "Missouri", "Indiana", "Oklahoma", "Wisconsin", "Nebraska", "Iowa", "Michigan"], 
"central" : ["North Dakota", "South Dakota", "Colorado", "Wyoming", "Kansas", "Texas", "Minnesota", "Montana"], 
"west" : ["Washington", "California", "Oregon", "Utah", "New Mexico", "Alaska", "Nevada", "Arizona", "Idaho"]
}
regions = {**{state: region  for region, v in regions.items() for state in v}}
y_train = y_train.map(regions)
assert y_train.isnull().sum() == 0

classifier = HistGradientBoostingClassifier(learning_rate = .2, min_samples_leaf = 10, l2_regularization = .05,   verbose =0, random_state = 12345)

I do a little feature engineering: 

I create a `temperature_mean` variable 
averaging high, mid, and low temperatures Then I 
use multivariate NaN imputation for the many missing values in the chosen predictors.

In [ ]:
X_train["temperature_mean"] = X_train[[ "temperature_high", "temperature_mid", "temperature_low" ]].sum(axis = 1) / 3
features =[ "humidity", "cloud_cover", "precip_intensity", "pressure", "visibility", "precip_probability", "wind_speed" ]
predictors = X_train[features]
imputer =  IterativeImputer(max_iter=10, random_state=12345)
imputer.fit(predictors)
predictors = imputer.transform(predictors)

Now to fit and evaluate the model. Cross-validated scores aren't very good; the 
level of accuracy obtained isn't much better than the naive classifier (i.e., 
predicting the most likely class for each observation).

But the accuracy on the training data is about 80%, very good. That suggests overfitting, 
so I'll try to make the model less flexible.

In [ ]:
model = classifier.fit(predictors, y_train)
scores = cross_val_score(classifier, predictors, y_train, cv = 5)
y_train.value_counts() /len(y_train)
classifier.score(predictors, y_train)

Let's try again, this time scaling predictors and reducing flexibility. This improves scores a little.

In [ ]:
scaled = np.apply_along_axis(lambda x: (x - np.mean(x))/ np.var(x), axis = 1, arr = predictors)
classifier = HistGradientBoostingClassifier(learning_rate = .03, max_leaf_nodes = 15, n_iter_no_change = 5, verbose = 0, random_state = 12345)
model = classifier.fit(predictors, y_train)
scores = cross_val_score(classifier, predictors, y_train, cv = 5)

A random forest fails to do better.

In [ ]:
forest = RandomForestClassifier(n_estimators = 200,   random_state = 12345,  ccp_alpha = 0.1)
model = forest.fit(predictors, y_train)
scores = cross_val_score(forest, predictors, y_train, cv = 5)
scores

Prediction time. The model again fares badly.

In [ ]:
y_test = y_test.map(regions)
X_test["temperature_mean"] = X_test[[ "temperature_high", "temperature_mid", "temperature_low" ]].sum(axis = 1) / 3
X_test = X_test[features]
X_test = imputer.transform(X_test)
X_test= np.apply_along_axis(lambda x: (x - np.mean(x))/ np.var(x), axis = 1, arr = X_test)
predictions = classifier.predict(X_test)
classifier.score(X_test, y_test)